In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
base_dir = "../input/celebrity-face-image-dataset/Celebrity Faces Dataset"
import tensorflow as tf
import pathlib
!pip install split-folders

**Split data as train and validation.**

In [ ]:
import splitfolders

def split_data(data_path):
    data = pathlib.Path(data_path)
    splitfolders.ratio(data,output = "Images/",seed = 42,ratio = (0.8,0.2),group_prefix = None)

In [ ]:
split_data("../input/celebrity-face-image-dataset/Celebrity Faces Dataset")

In [ ]:
"""All images will be scaled 1./255 to obtain 0-1 normalized image.Also use data augmentation."""
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,rotation_range = 40,
                                                                width_shift_range = 0.2,height_shift_range = 0.2,
                                                                shear_range = 0.2,zoom_range = 0.2,horizontal_flip=True,
                                                                vertical_flip = True,fill_mode = "nearest",
                                                                )
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory("Images/train",target_size = (150,150),class_mode = "categorical",
                                                   batch_size =64,subset = "training",seed = 42)

validation_generator = validation_datagen.flow_from_directory("Images/val",target_size = (150,150),
                                                    class_mode = "categorical",batch_size = 64,
                                                             seed = 42)


**CNN ARCHITECTURE WITH TRANSFER LEARNING**

**(1)Use Xception as base model.**

In [ ]:
from tensorflow.keras.applications.xception import Xception
base_model = Xception(input_shape = (150,150,3),include_top = False,weights = "imagenet",pooling = "max")
"""Freeze all layers to stop updating weights of imagenet."""
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
base_model.summary()

**(2)Create an architecture to feed models.**

In [ ]:
"""Here we can get add_11 as last layer.It means that we can start updating our weights after add_11 layer."""
last_layer = base_model.get_layer("add_11")
print(last_layer.output_shape)

In [ ]:
classes = os.listdir(base_dir)
"""Flatten layer to reduce input dim to 1D."""
x = tf.keras.layers.BatchNormalization()(last_layer.output)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
"""Add fully connected layers with 256 units."""
x = tf.keras.layers.Dense(units = 128,activation = "relu")(x)
"""Add dropout layer."""
x = tf.keras.layers.Dropout(0.5)(x)
"""Output Layer."""
x = tf.keras.layers.Dense(len(classes),activation = "softmax")(x)
"""Here we can connect our model end to end."""
model = tf.keras.models.Model(base_model.input,x)

In [ ]:
model.summary()

**COMPILE AND FIT MODEL.**

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),loss = "categorical_crossentropy",
             metrics = ["acc"])
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs = {}):
        if logs.get("acc") - logs.get("val_acc") > 0.1:
            print("Model tends to be overfitting.Stop Training")
            self.model.stop_training = True
callback = myCallback()

In [ ]:
history = model.fit(train_generator,epochs = 50,batch_size = 64,validation_data = validation_generator,
                   callbacks = [callback],verbose = 1)

**PLOT LOSS AND ACCURACY**

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["acc"]
val_acc = history.history["val_acc"]
loss = history.history["loss"];
val_loss = history.history["val_loss"]
epochs = range(0,len(history.history["loss"]))


In [ ]:
plt.plot(epochs,acc,label = "Training Accuracy")
plt.plot(epochs,val_acc,label = "Validation Accuracy")
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs,loss,label = "Training Loss")
plt.plot(epochs,val_loss,label = "Validation Loss")
plt.legend()
plt.show()

**SAVE MODEL AND LOAD MODEL**

In [ ]:
model.save("celebrity_classification_TFL.h5")

In [ ]:
model1 = tf.keras.models.load_model("celebrity_classification_TFL.h5")

**GET THE IMAGES FROM INTERNET AND PROCESS THEM.**

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import numpy as np

In [ ]:
def get_and_process(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img1 = img
    """Resize image to appropriate shape for model."""
    img = img.resize((150,150))
    """Convert img to numpy array,rescale it,expand dims and check vertically."""
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = x / 255.0
    x = np.expand_dims(x,axis = 0)
    img_tensor = np.vstack([x])
    return img1,img_tensor
    
    
    

**FINAL :PREDICT IMAGE**

In [ ]:
import matplotlib.pyplot as plt 
url = "https://ichef.bbci.co.uk/news/640/cpsprodpb/8012/production/_124368723_gettyimages-1357401691.jpg"
img_1,test_img = get_and_process(url)
pred = model1.predict(test_img)
classes = list(train_generator.class_indices.keys())
print(f"Prediction is : {classes[np.argmax(pred)]}")
plt.imshow(img_1)
plt.show()

print(classes)
print(pred)